In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd
from tqdm.notebook import tqdm
import random
import numpy as np
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
# 장르별 base 주소
ballad = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&dtlGnrCode=#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
dance = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
hiphop = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0300#params%5BgnrCode%5D=GN0300&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
RB = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0400#params%5BgnrCode%5D=GN0400&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
indi = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0500#params%5BgnrCode%5D=GN0500&params%5BdtlGnrCode%5D=GN0501&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
rock = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0600#params%5BgnrCode%5D=GN0600&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
trot = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0700#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=GN0701&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="
folk = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0800#params%5BgnrCode%5D=GN0800&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex="

In [ ]:
start = 1 #첫번째로 크롤링할 곡 번호(순위) 
num_of_data = 15000 #크롤링하고 싶은 데이터 갯수 
end = start + (num_of_data-50) + 1 #마지막으로 크롤링할 곡 번호(순위) 

In [ ]:
genre_dict = {ballad:'ballad', dance:'dance', hiphop:'rap/hiphop', RB: 'R&B/Soul', indi:'indi', 
             rock:'rock/metal', trot:'trot', folk:'folk/blues'}
genre_dict[hiphop]

In [ ]:
df=pd.DataFrame(columns=['제목', '곡 번호'])

In [ ]:
def crawling(num_of_data, genre, start, end, df):
  for i in tqdm(range(start, end, 50)):
    driver = webdriver.Chrome('chromedriver', options=options) 
    url = genre + str(i)
    driver.get(url)
    #4초 쉬어주기
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    #html.parser를 사용해서 soup에 넣겠다
      
    #노래제목
    song_title = []
    #곡 번호
    song_number = []

    title = soup.find_all("div",{"class":"ellipsis rank01"})  

    for j in title:
      if j.find('span').find('span'):
        song_title.append(j.find('a').text)  #노래 제목 저장
        text = j.find('a')['href']
        song_number.append(re.search(r'[,](\d+)[)]', text).group(1)) # 곡 번호 저장
    
    #중간저장
    new = pd.DataFrame(np.array([song_title, song_number]).T, columns=['제목', '곡 번호'])
    df = pd.concat([df, new], ignore_index=True)
    df.to_csv('/content/drive/Shareddrives/Parrot/주소', encoding='utf-8')
    print(str(i)+'부터 50개 저장완료')
    #2초 내에서 랜덤으로 쉬어주기
    time.sleep(0.5)
  
  return df




In [ ]:
crawling(num_of_data, hiphop, start, end, df)